# Multivariate linear regression
using python package statsmodels

In [1]:
import statsmodels.api as sm
import pandas as pd

Preparing Your Data: Make sure your data is in a suitable format. Typically, you'll have a pandas DataFrame with one column as the dependent variable (the outcome you're interested in predicting) and the other columns as independent variables (the predictors).

In [23]:
DATA_FOLDER = "../data/final/"
ihd_df = pd.read_csv(DATA_FOLDER + "gbd_IschemicHeartDisease_DeathsIncidence.csv")
ihd_df.rename(columns={'year': 'Year'}, inplace=True)
ihd_df = ihd_df[ihd_df['measure_name'] == 'Incidence']
ihd_df = ihd_df[ihd_df["location_name"] == "Germany"].copy()
ihd_df.head()

,measure_name,location_name,cause_name,Year,Value,upper,lower
7984,Incidence,Germany,Ischemic heart disease,1990,770.782550,830.581487,715.950617
7985,Incidence,Germany,Ischemic heart disease,1991,766.235116,822.822332,713.701355
7986,Incidence,Germany,Ischemic heart disease,1992,757.466390,811.731442,706.520709
7987,Incidence,Germany,Ischemic heart disease,1993,746.615774,797.680308,697.650558
7988,Incidence,Germany,Ischemic heart disease,1994,738.484981,789.628450,689.209094


In [22]:
fat_df = pd.read_csv(DATA_FOLDER + "daily_per_capita_fat_supply_final.csv")
fat_df = fat_df[fat_df["Country Name"] == "Germany"].copy()
fat_df.head()

,Country Name,Country Code,Series Name,Year,Value
4263,Germany,DEU,Fat consumption per day per capita (grams),1961,113.313324
4264,Germany,DEU,Fat consumption per day per capita (grams),1962,117.463394
4265,Germany,DEU,Fat consumption per day per capita (grams),1963,116.231660
4266,Germany,DEU,Fat consumption per day per capita (grams),1964,118.172020
4267,Germany,DEU,Fat consumption per day per capita (grams),1965,120.793110


In [37]:
# alcohol_df = pd.read_csv(DATA_FOLDER + "wdi_AlcoholConsumption.csv")
# alcohol_df = alcohol_df[alcohol_df["Country Name"] == "Germany"].copy()
# alcohol_df.head()
alcohol_df = pd.read_csv("../data/raw/alcohol_germany.csv")
alcohol_df = alcohol_df[alcohol_df["LOCATION"] == "DEU"].copy()
alcohol_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
454,DEU,ALCOHOL,TOT,LT_CAP15,A,1961,11.0,NaN
455,DEU,ALCOHOL,TOT,LT_CAP15,A,1962,12.1,NaN
456,DEU,ALCOHOL,TOT,LT_CAP15,A,1963,12.9,NaN
457,DEU,ALCOHOL,TOT,LT_CAP15,A,1964,13.7,NaN
458,DEU,ALCOHOL,TOT,LT_CAP15,A,1965,14.0,NaN


In [45]:
# smoking_df = pd.read_csv(DATA_FOLDER + "share-of-adults-who-smoke.csv")
# smoking_df = smoking_df[smoking_df["Entity"] == "Germany"].copy()
# smoking_df.head()
smoking_df = pd.read_csv("../data/raw/smoking_germany.csv")
smoking_df = smoking_df[smoking_df["LOCATION"] == "DEU"].copy()
smoking_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
207,DEU,SMOKERS,TOT,PC_POP15,A,1978,28.5,NaN
208,DEU,SMOKERS,TOT,PC_POP15,A,1989,25.1,NaN
209,DEU,SMOKERS,TOT,PC_POP15,A,1992,24.8,B
210,DEU,SMOKERS,TOT,PC_POP15,A,1995,24.3,NaN
211,DEU,SMOKERS,TOT,PC_POP15,A,1999,24.7,NaN


In [46]:
combined_df = ihd_df[['Value', 'Year']].copy()
combined_df.rename(columns={"Value":"IHD_incidence"}, inplace=True)

fat_df_renamed = fat_df.rename(columns={'Value': 'fat_consumption'})
combined_df = combined_df.merge(fat_df_renamed[['fat_consumption', 'Year']], on='Year', how='left')

alcohol_df_renamed = alcohol_df.rename(columns={'Value': 'alcohol_consumption', "TIME": "Year"})
combined_df = combined_df.merge(alcohol_df_renamed[['alcohol_consumption', 'Year']], on='Year', how='left').dropna()

alcohol_df_renamed = alcohol_df.rename(columns={'Value': 'daily_smokers', "TIME": "Year"})
combined_df = combined_df.merge(alcohol_df_renamed[['daily_smokers', 'Year']], on='Year', how='left').dropna()

combined_df.drop(columns=['Year'], inplace=True)
# combined_df.reset_index(drop=True, inplace=True)
combined_df

,IHD_incidence,fat_consumption,alcohol_consumption,daily_smokers
0,770.782550,142.51752,14.9,14.9
1,766.235116,143.16267,13.9,13.9
2,757.466390,148.68661,13.8,13.8
3,746.615774,140.65509,13.5,13.5
4,738.484981,141.04372,13.4,13.4
5,729.925213,141.60830,13.4,13.4
6,711.961594,148.14981,13.1,13.1
7,683.596598,146.78265,13.0,13.0
8,652.358799,147.06120,12.7,12.7
9,627.604773,144.04678,12.8,12.8


Fitting the Model:

In [51]:
# Assuming 'df' is your DataFrame and 'y' is the dependent variable
X = combined_df.drop('IHD_incidence', axis=1)  # Independent variables
y = combined_df['IHD_incidence']  # Dependent variable

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

In [52]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          IHD_incidence   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.905
Method:                 Least Squares   F-statistic:                     138.5
Date:                Wed, 17 Jan 2024   Prob (F-statistic):           6.39e-15
Time:                        15:18:16   Log-Likelihood:                -137.73
No. Observations:                  30   AIC:                             281.5
Df Residuals:                      27   BIC:                             285.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                -390.9167    